# Reconstruction exmaple

## Import the relevant libraries

In [2]:
pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00
INFO: pip is looking a

In [3]:
pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 12.8 MB/s eta 0:00:00


In [5]:
pip install sympy

In [6]:
pip install qbraid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.0/524.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 4.5 MB/s eta 0:00:00


In [7]:
from qbraid.api import QbraidSession

session = QbraidSession(api_key='7m4dptrqcig')
session.save_config()

In [8]:
import cirq
import numpy as np
import sympy
from qiskit import *
from qiskit import QuantumCircuit

In [11]:
from typing import Any, Dict, Optional

from qbraid.api import QbraidSession
from qbraid.exceptions import QbraidError
from qbraid.transpiler import convert_to_package


class Server:
    """Server class containing the attack() and probe() methods for IonQ challenge."""

    def __init__(self, qbraid_session: Optional[QbraidSession] = None):
        """Initialize the server class with a QbraidSession object."""
        self._session = qbraid_session or QbraidSession()

    @property
    def session(self) -> QbraidSession:
        """Return the QbraidSession object."""
        return self._session

    @staticmethod
    def _verify_qasm_program(qasm: str) -> None:
        """Verify that the qasm program is valid."""
        try:
            _ = convert_to_package(qasm, "cirq")
        except QbraidError as err:
            raise ValueError("Invalid OpenQASM program.") from err

    def _post_request(
        self, action: str, vault_index: int, circuit: str
    ) -> Dict[str, Any]:
        """Send a request to a specified endpoint with given parameters."""
        self._verify_qasm_program(circuit)
        query = {"vaultIndex": vault_index, "openQasm": circuit}
        resp = self.session.post(f"/iquhack/ionq/{action}", json=query)
        return resp.json()

    def attack(self, vault_index: int, circuit: str) -> Dict[str, float]:
        """Returns score."""
        resp_data = self._post_request("attack", vault_index, circuit)
        score_keys = {"rawScore", "costFactor", "score"}
        score_dict = {k: resp_data[k] for k in score_keys if k in resp_data}
        return score_dict

    def probe(self, vault_index: int, circuit: str) -> Dict[str, float]:
        """Returns histogram data with keys in the big-endian decimal
        representation of measurement bit strings."""
        resp_data = self._post_request("probe", vault_index, circuit)
        return resp_data["histogram"]

    def state(self) -> Dict[str, Any]:
        """Returns user challenge metadata."""
        return self.session.get("/iquhack/ionq/state").json()

    def register(self, team_name: str) -> str:
        """Initialize session. Only needs to be done once."""
        resp = self.session.post(
            f"/iquhack/ionq/register", json={"teamName": team_name}
        ).json()
        message = resp["message"]
        for ln in message.split("\n"):
            print(ln.strip())

server = Server()

## Test probe using vault 0

In [12]:
cq = cirq.Circuit()
cq.append(cirq.H.on(cirq.LineQubit(0)))
cq.append(cirq.H.on(cirq.LineQubit(1)))
cq.append(cirq.H.on(cirq.LineQubit(2)))
test_probe = cirq.qasm(cq)
print(test_probe)

// Generated from Cirq v1.3.0

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [q(0), q(1), q(2)]
qreg q[3];


h q[0];
h q[1];
h q[2];



In [14]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!Be careful about the Endian of returned histogram!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
server.probe(vault_index=0, circuit=test_probe)

{'0': 0.25, '3': 0.22, '5': 0.32, '6': 0.21}

## Test attack using vault 0

In [15]:
cq = cirq.Circuit()
cq.append(cirq.CNOT.on(cirq.LineQubit(0), cirq.LineQubit(2)))
cq.append(cirq.CNOT.on(cirq.LineQubit(0), cirq.LineQubit(1)))
cq.append(cirq.H.on(cirq.LineQubit(0)))
test_attack = cirq.qasm(cq)
print(test_attack)

// Generated from Cirq v1.3.0

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [q(0), q(1), q(2)]
qreg q[3];


cx q[0],q[2];
cx q[0],q[1];
h q[0];



In [16]:
server.attack(vault_index=0, circuit=test_attack)

{'costFactor': 0.8, 'score': 0.8, 'rawScore': 1}

## Check state

Check state for your team's current score(s), and number of probes/attacks remaining

In [17]:
server.state()

{'team': 'QuanTeam',
 'totalAvgScore': 0.006370937712059497,
 'topVaultScores': [0.8,
  0.0014516129032258066,
  0.023241758241758243,
  0,
  0.0004962406015037594,
  0.03409090909090909,
  0,
  0,
  0,
  0.016,
  0.0011707317073170733,
  0,
  0],
 'probesRemaining': [39, 19, 19, 16, 17, 17, 20, 20, 20, 19, 19, 20, 20],
 'attacksRemaining': [37, 19, 19, 15, 15, 18, 20, 20, 20, 19, 19, 20, 20],
 'reminder': 'Vault index 0 is a test vault, and does not count towards your final score.'}